In [17]:
from pathlib import Path
import pandas as pd
from typing import Union

ICS_CSV = r"C:\Users\dkerr\Documents\GISRede\OXFORD_UNI_WORK\ICS_Analysis\src\dashboard\api\app\data\db-data\ICS_DATABASE_TABLE.csv"
FUNDERS_TABLE = r'C:\Users\dkerr\Documents\GISRede\OXFORD_UNI_WORK\ICS_Analysis\data\dashboard\funders.csv'
LOOKUP_CSV = 'LOOKUP.csv'
FUNDERS_CSV = 'FUNDERS.csv'
df_ics = pd.read_csv(ICS_CSV)
df_funders = pd.read_csv(FUNDERS_TABLE)
df_funders = df_funders[['REF impact case study identifier', 'Funders[full name]']]
df_funders.rename(columns={"REF impact case study identifier": 'ics_id', 'Funders[full name]': 'funder'}, inplace=True)
df_funders.dropna(subset='ics_id', inplace=True)
df_ics = df_ics[['id', 'ics_id']]
df_ics.set_index('ics_id', inplace=True)
df_funders.set_index('ics_id', inplace=True)
display(df_ics.head(2))
display(df_funders.head(2))

,id
ics_id,
1e6e075e-d5f5-421c-bb45-2708636b5190,0
73be8e7c-907e-4e5b-8a8b-953b4d394c1b,1


,funder
ics_id,
1e6e075e-d5f5-421c-bb45-2708636b5190,European Union [EU]; Economic and Social Resea...
73be8e7c-907e-4e5b-8a8b-953b4d394c1b,Innovate UK [GBR]; Biotechnology and Biologica...


In [31]:
import numpy as np
df_join = df_funders.join(df_ics)
def transform_funders(row):
    if str(row.funder) != 'nan':
        return row.funder.split(';')
    return None
df_join['funders_list'] = df_join.apply(transform_funders, axis=1)
df_join.reset_index(inplace=True)
df_join = df_join[['id', 'funders_list']]
df_lookup = df_join.explode('funders_list')
df_lookup.rename(columns={'id': 'ics_table_id', 'funders_list': 'funder'}, inplace=True)
df_lookup['id'] = df_lookup.index.copy().astype(int)
df_lookup

,ics_table_id,funder,id
0,0,European Union [EU],0
0,0,Economic and Social Research Council [GBR],0
0,0,Arts and Humanities Research Council [GBR],0
0,0,Engineering and Physical Sciences Research Co...,0
0,0,Nuffield Foundation,0
...,...,...,...
6360,6360,Wellcome Trust,6360
6360,6360,European Commission [EU],6360
6360,6360,Leona M and Harry B Helmsley Charitable Trust,6360
6360,6360,Engineering and Physical Sciences Research Co...,6360


In [32]:
len(df_lookup.funder.unique())

3707